In [1]:
%load_ext autoreload
%autoreload 2
import os 
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.85'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from nn_ansatz import *

import jax
import jax.random as rnd
import jax.numpy as jnp
from jax import vmap, jit, grad, pmap
from jax.experimental.optimizers import adam
from jax import tree_util
from tqdm.notebook import trange
import math
from functools import partial
import itertools
import ase

In [6]:
from ase.build import bulk
x = bulk('H', orthorhombic=True)

ValueError: No suitable reference data for bulk H.  Reference data: {'symmetry': 'diatom', 'd': 0.74}

In [3]:




cfg = setup(system='LiSolidBCC',
            n_pre_it=0,
            n_walkers=512,
            n_layers=2,
            n_sh=64,
            step_size=0.05,
            n_ph=32,
            scalar_inputs=False,
            orbitals='anisotropic',
            n_periodic_input=1,
            opt='adam',
            n_det=4,
            print_every=50,
            save_every=2500,
            lr=1e-4,
            n_it=30000,
            debug=True,
            distribute=False)

logger = Logging(**cfg)

keys = rnd.PRNGKey(cfg['seed'])
if bool(os.environ.get('DISTRIBUTE')) is True:
    keys = rnd.split(keys, cfg['n_devices']).reshape(cfg['n_devices'], 2)

mol = SystemAnsatz(**cfg)

owf = create_wf(mol, orbitals=True)
vwf = create_wf(mol)
params = initialise_params(mol, keys)

sampler = create_sampler(mol, vwf)

walkers = None
if walkers is None:
    walkers = generate_walkers_around_nuclei(mol.n_el_atoms, mol.atom_positions, mol.n_walkers)
elif not len(walkers) == mol.n_walkers:
    n_replicate = math.ceil(mol.n_walkers / len(walkers))
    walkers = jnp.concatenate([walkers for i in range(n_replicate)], axis=0)
    walkers = walkers[:mol.n_walkers, ...]

le = create_local_kinetic_energy(vwf)
pe = create_potential_energy(mol)



if bool(os.environ.get('DISTRIBUTE')) is True:
    walkers = walkers.reshape(mol.n_devices, -1, *walkers.shape[1:])
    le = pmap(le, in_axes=(None, 0))
    pe = pmap(pe, in_axes=(0, None, None))

if mol.pbc:
    print('sampling no infs, this could take a while')
    walkers = keep_in_boundary(walkers, mol.basis, mol.inv_basis)

def logabssumdet(orb_up: jnp.array, orb_down: jnp.array) -> jnp.array:
    s_up, log_up = jnp.linalg.slogdet(orb_up)
    s_down, log_down = jnp.linalg.slogdet(orb_down) if not orb_down is None else (jnp.ones_like(s_up), jnp.zeros_like(log_up))

    # logdet_sum = jnp.where(~jnp.isinf(log_up), log_up, -jnp.ones_like(log_up)*10.**10) + jnp.where(~jnp.isinf(log_down), log_down, -jnp.ones_like(log_down)*10.**10)
    logdet_sum = log_up + log_down
    logdet_max = jnp.max(logdet_sum)

    argument = s_up * s_down * jnp.exp(logdet_sum - logdet_max)
    sum_argument = jnp.sum(argument, axis=0)
    sign = jnp.sign(sum_argument)

    return jnp.log(jnp.abs(sum_argument)) + logdet_max, sign




version 		 261021
seed 		 369
n_devices 		 1
save_every 		 2500
print_every 		 50
exp_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/261021/junk/adam_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det4/run6
events_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/261021/junk/adam_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det4/run6/events
models_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/261021/junk/adam_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det4/run6/models
opt_state_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/261021/junk/adam_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det4/run6/models/opt_state
pre_path 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/pretrained/s64_p32_l2_det4_1lr-4_i0.pk
timing_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/261021/junk/adam_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det4/run6/events/timing
csv_cfg_path 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/261021/junk/adam_1lr-4_1d-3_1n

In [4]:
def check(x):
    return jnp.isnan(x).any() or jnp.isinf(x).any()

if bool(os.environ.get('DISTRIBUTE')) is True:
    pwf = pmap(vwf, in_axes=(None, 0))
else:
    pwf = vwf


compute_single_stream_vectors = vmap(partial(compute_single_stream_vectors_i, basis=mol.basis, inv_basis=mol.inv_basis, pbc=True), in_axes=(0, None))




In [5]:
print(walkers.shape, mol.r_atoms.shape)
log_psi = pwf(params, walkers)

print(check(log_psi))
print(log_psi[idx])

(512, 6, 3) (2, 3)
False


NameError: name 'idx' is not defined

In [16]:
walkers = jnp.squeeze(load_pk('walkers.pk'), axis=0)
print(walkers.shape)
e_locs = load_pk('e_locs.pk')
idx = jnp.where(jnp.isnan(e_locs))[0][0]
print(idx)
print(walkers[idx])
print(mol.r_atoms)
ss_vecs = compute_single_stream_vectors_i(walkers[idx], mol.r_atoms, basis=mol.basis, inv_basis=mol.inv_basis, pbc=True)
print(ss_vecs / 6.63)

(512, 6, 3)
67
[[2.8447561  4.448132   4.4388056 ]
 [6.1663704  6.232988   0.23771067]
 [3.3156629  3.3906832  3.3915722 ]
 [2.7414308  3.0236557  3.9150403 ]
 [6.0261087  1.3382857  6.3388243 ]
 [2.7968063  3.8667147  2.8075783 ]]
[[0.    0.    0.   ]
 [3.315 3.315 3.315]]
[[[-4.29073334e-01  3.29090208e-01  3.30496937e-01]
  [ 7.09266886e-02 -1.70909807e-01 -1.69503108e-01]]

 [[ 6.99290708e-02  5.98811843e-02 -3.58537994e-02]
  [-4.30070966e-01 -4.40118849e-01  4.64146227e-01]]

 [[ 4.99900043e-01  4.88584787e-01  4.88450676e-01]
  [-9.99703916e-05 -1.14152515e-02 -1.15493489e-02]]

 [[-4.13488805e-01 -4.56056684e-01  4.09496218e-01]
  [ 8.65112096e-02  4.39433493e-02 -9.05038044e-02]]

 [[ 9.10846740e-02 -2.01853052e-01  4.39179279e-02]
  [-4.08915341e-01  2.98146963e-01 -4.56082106e-01]]

 [[-4.21841085e-01  4.16785151e-01 -4.23465818e-01]
  [ 7.81589374e-02 -8.32148865e-02  7.65342042e-02]]]


In [ ]:
-1./(8.*(1. + 2.*orb_vector_shifted)

In [9]:


step_size = mol.step_size
# walkers *= 0.
for i in range(1000):
    if i % 1 == 0:
        print(i)
    keys, subkeys = key_gen(keys)
    walkers, _, step_size = sampler(params, walkers, subkeys, step_size)
    # ke = le(params, walkers)
    ke = pe(walkers, mol.r_atoms, mol.z_atoms)
    x = pwf(params, walkers)  
    check_ke = check(ke)  
    isnan = jnp.isnan(x).any()
    isinf = jnp.isinf(x).any()
    if isnan or isinf or check_ke:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
print(ke)
print(jnp.diag(mol.basis[0]))

[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf in

In [ ]:
s_up, log_up = jnp.linalg.slogdet(orb_up)
jnp.where(jnp.isinf(log_up))

(DeviceArray([378, 378, 378, 378], dtype=int32),
 DeviceArray([0, 1, 2, 3], dtype=int32))

In [ ]:
jnp.isinf(log_up[378])

DeviceArray([ True,  True,  True,  True], dtype=bool)

In [ ]:
print(log_up.shape)

(512, 4)


In [ ]:
print(orb_up[378])
print(walkers[378])

[[[-0.0000000e+00 -0.0000000e+00 -0.0000000e+00]
  [ 8.1305001e-03  7.8869974e-03  3.0328596e-03]
  [-3.1967011e-01  1.0195731e-01 -7.0489198e-01]]

 [[-0.0000000e+00 -0.0000000e+00 -0.0000000e+00]
  [ 5.1662428e-03  5.8913063e-03  7.2862888e-03]
  [-3.5429075e+00 -1.7007575e+00 -1.2084190e+00]]

 [[-0.0000000e+00 -0.0000000e+00  0.0000000e+00]
  [-1.7099148e-03  8.6398149e-04  6.1524189e-03]
  [ 4.3009886e-01  1.4784889e+00  3.4972703e+00]]

 [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 7.9447906e-03  2.8494976e-03  7.2592115e-03]
  [-4.1527462e-01  1.3134570e+00  2.4878182e+00]]]
[[-2.6156468e-02 -1.1797682e+01 -3.3316281e+00]
 [ 7.1275134e+00  9.1380510e+00  3.7403147e+00]
 [ 1.5415110e+01  2.9115196e+01  1.8620895e+01]
 [ 2.7590673e+01  1.7976408e+01  1.9681137e+01]
 [-7.9012446e+00 -3.7934704e+00  1.6503401e+00]
 [ 1.9020798e+01  1.6751478e+01  9.2529478e+00]]


In [ ]:
test_walkers = keep_in_boundary(walkers, mol.basis, mol.inv_basis)
print(test_walkers[378])

[[-1.73417389e-01 -7.82186279e+01 -2.20886955e+01]
 [ 4.72554131e+01  6.05852776e+01  2.47982864e+01]
 [ 1.02202179e+02  1.93033752e+02  1.23456535e+02]
 [ 1.82926163e+02  1.19183586e+02  1.30485947e+02]
 [-5.23852539e+01 -2.51507092e+01  1.09417553e+01]
 [ 1.26107895e+02  1.11062302e+02  6.13470459e+01]]


In [ ]:
def transform_vector_space(vectors: jnp.array, basis: jnp.array) -> jnp.array:
    '''
    case 1 catches non-orthorhombic cells 
    case 2 for orthorhombic and cubic cells
    '''
    if basis.shape == (3, 3):
        return jnp.dot(vectors, basis)
    else:
        return vectors * basis


def keep_in_boundary(walkers, basis, inv_basis):
    talkers = transform_vector_space(walkers, inv_basis)
    talkers = jnp.fmod(talkers, 1.)
    talkers = jnp.where(talkers < 0., talkers + 1., talkers)
    talkers = transform_vector_space(talkers, basis)
    return talkers

print(mol.inv_basis)
test_walkers = transform_vector_space(walkers, mol.inv_basis)
test_talkers = jnp.fmod(test_walkers, 1.)
test_talkers1 = jnp.where(test_talkers < 0., test_talkers + 1., test_talkers)

[[0.15082955 0.15082955 0.15082955]]


In [ ]:
print(test_walkers[378])
print(test_talkers[378])
print(test_talkers1[378])

[[-3.9451686e-03 -1.7794391e+00 -5.0250798e-01]
 [ 1.0750396e+00  1.3782881e+00  5.6414998e-01]
 [ 2.3250542e+00  4.3914323e+00  2.8085814e+00]
 [ 4.1614890e+00  2.7113736e+00  2.9684970e+00]
 [-1.1917412e+00 -5.7216746e-01  2.4892005e-01]
 [ 2.8688984e+00  2.5266180e+00  1.3956180e+00]]
[[-0.00394517 -0.7794391  -0.502508  ]
 [ 0.07503963  0.37828815  0.56415   ]
 [ 0.32505417  0.3914323   0.80858135]
 [ 0.16148901  0.71137357  0.96849704]
 [-0.19174123 -0.57216746  0.24892005]
 [ 0.8688984   0.526618    0.39561796]]
[[0.9960548  0.22056091 0.49749202]
 [0.07503963 0.37828815 0.56415   ]
 [0.32505417 0.3914323  0.80858135]
 [0.16148901 0.71137357 0.96849704]
 [0.8082588  0.42783254 0.24892005]
 [0.8688984  0.526618   0.39561796]]


In [ ]:
print(mol.basis)

[[6.63 6.63 6.63]]
